In [1]:
import numpy as np
import pandas as pd
 
import pandas as pd
from pandas import Series, DataFrame 
from tensorflow.python.framework import ops
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import sklearn.metrics as skm
%matplotlib inline
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
plt.rcParams['figure.figsize'] = (7.0, 4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import ensemble,metrics,cross_validation

C:\Users\zhaoy\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
id_test = test['id'].values
target_train = train['target'].values
train = train.drop(['target','id'], axis = 1)
test = test.drop(['id'], axis = 1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)
test = test.drop(col_to_drop, axis=1)  
train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)
cat_features = [a for a in train.columns if a.endswith('cat')]
for column in cat_features:
    temp = pd.get_dummies(pd.Series(train[column]))
    train = pd.concat([train,temp],axis=1)
    train = train.drop([column],axis=1)
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    test = pd.concat([test,temp],axis=1)
    test = test.drop([column],axis=1)
test=test.fillna(method='pad')
train=train.fillna(method='pad')
print(train.values.shape, test.values.shape)
print(target_train.shape)

(595212, 198) (892816, 198)
(595212,)


In [4]:
def gini(y, pred):
    fpr, tpr, thr = metrics.roc_curve(y, pred, pos_label=1)
    g = 2 * metrics.auc(fpr, tpr) -1
    return g

In [46]:
def swap_data(data):
    data_swap = []
    for row in data:
        swap_col_idxs = np.random.choice(data.shape[1], 33)
        swap_row_idx = np.random.choice(data.shape[0], 1)[0]
        new_row = row.copy()
        new_row[swap_col_idxs] = data[swap_row_idx][swap_col_idxs]
        data_swap.append(new_row)
    return np.array(data_swap)

In [5]:
x_train ,x_test, y_train, y_test = train_test_split(train, target_train, test_size=0.2, random_state=1)

log_model=LogisticRegression()
log_model.fit(x_train,y_train)
pred_log = log_model.predict_proba(x_test)[:,1]
print( "Gini log  Test= ", gini(y_test, pred_log))

Gini log  Test=  0.257704953635


In [6]:
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [58]:
input_layer = Input(shape=(train.shape[1],))

encoded = Dense(1000, activation='relu')(input_layer)
encoded = Dense(500, activation='linear')(encoded)

decoded = Dense(1000, activation='relu')(encoded)
decoded = Dense(train.shape[1], activation='linear')(decoded)

In [59]:
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

In [60]:
encoder = Model(input_layer, encoded)

In [61]:
for i in range(50):
    data_swap = swap_data(train.values)
    autoencoder.fit(data_swap, train.values,
                    epochs=1,
                    batch_size=512,
                    shuffle=True)

Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0465     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0407     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0397     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0390     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0387     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0383     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0381     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0378     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0376     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0374     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0371     
Epoch 1/1
595212/595212 [==============================] - 9s - loss: 0.0370     
Epoch 1/1
595212

In [62]:
train_encoded = encoder.predict(train.values)
train_encoded.shape

(595212, 500)

In [63]:
x_train ,x_test, y_train, y_test = train_test_split(train_encoded, target_train, test_size=0.2, random_state=1)

log_model=LogisticRegression()
log_model.fit(x_train,y_train)
pred_log = log_model.predict_proba(x_test)[:,1]
print( "Gini log  Test= ", gini(y_test, pred_log))

Gini log  Test=  0.28134335296


In [64]:
x_train ,x_test, y_train, y_test = train_test_split(train_encoded, target_train, test_size=0.2, random_state=0)

log_model=LogisticRegression()
log_model.fit(x_train,y_train)
pred_log = log_model.predict_proba(x_test)[:,1]
print( "Gini log  Test= ", gini(y_test, pred_log))

Gini log  Test=  0.275363902522


In [65]:
x_train ,x_test, y_train, y_test = train_test_split(train_encoded, target_train, test_size=0.2, random_state=2)

log_model=LogisticRegression()
log_model.fit(x_train,y_train)
pred_log = log_model.predict_proba(x_test)[:,1]
print( "Gini log  Test= ", gini(y_test, pred_log))

Gini log  Test=  0.269428439654


In [66]:
x_train ,x_test, y_train, y_test = train_test_split(train_encoded, target_train, test_size=0.2, random_state=3)

log_model=LogisticRegression()
log_model.fit(x_train,y_train)
pred_log = log_model.predict_proba(x_test)[:,1]
print( "Gini log  Test= ", gini(y_test, pred_log))

Gini log  Test=  0.282421169964


In [71]:
np.save("train_dae_encoded.npy", train_encoded)

In [75]:
autoencoder.save("denoising_autoencoder.h5")